In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import joblib # This is for saving our trained model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

print("--- Model Training Script Started ---")

# --- Step 1: Load the Dataset ---
print("Loading 'upi_transactions.csv'...")
try:
    df = pd.read_csv('upi_transactions.csv')
    # Convert timestamp column back to datetime objects
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    print("...Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'upi_transactions.csv' not found. Please ensure it is in the same folder.")
    # Stop execution if the file doesn't exist

# --- Step 2: Feature Engineering ---
# Machine learning models work with numbers. We will create numerical features 
# that help the model understand the data.

print("Creating new features...")

# Create a 'hour_of_day' feature from the timestamp
# This is very important for detecting fraud at unusual times
df['hour_of_day'] = df['timestamp'].dt.hour

print("...'hour_of_day' feature created.")
print("\n--- Data Preview with New Feature ---")
# We select only the numerical columns our model will use
# For this project, 'amount' and 'hour_of_day' are our key features.
features = ['amount', 'hour_of_day']
X = df[features]
y = df['is_fraud']

# Display the first few rows of our feature set
print(X.head())

# Display a summary of our data
print("\n--- Data Summary ---")
print(f"Total data points: {len(df)}")
print(f"Features for model: {features}")
print(f"Number of fraudulent transactions in data: {y.sum()}")


--- Model Training Script Started ---
Loading 'upi_transactions.csv'...
...Dataset loaded successfully.
Creating new features...
...'hour_of_day' feature created.

--- Data Preview with New Feature ---
   amount  hour_of_day
0  370.46           12
1  215.79           16
2  127.39           20
3  105.29            4
4  198.03           10

--- Data Summary ---
Total data points: 1040
Features for model: ['amount', 'hour_of_day']
Number of fraudulent transactions in data: 40


In [2]:
# --- Imports and Data Loading ---
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

# ADDED THIS SECTION TO FIX THE 'X not defined' ERROR
# This ensures our script always has the data it needs to run.
print("Loading data and creating features...")
try:
    df = pd.read_csv('upi_transactions.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour_of_day'] = df['timestamp'].dt.hour
    
    features = ['amount', 'hour_of_day']
    X = df[features]
    y = df['is_fraud']
    print("...Data loaded successfully.")
except FileNotFoundError:
    print("FATAL ERROR: 'upi_transactions.csv' not found. Please run the data generation notebook first.")
    # In a real script, you'd exit. Here we'll stop by not running the rest.


# --- Step 3: Train the Anomaly Detection Model ---

# The 'contamination' parameter is the most important one.
# It tells the model what proportion of the data is expected to be anomalous (fraudulent).
# We calculate it dynamically now.
contamination_rate = y.mean() # The mean of a binary (0/1) column is the proportion of 1s
model = IsolationForest(n_estimators=100, contamination=contamination_rate, random_state=42)

print("\nTraining the Isolation Forest model...")

# We train the model on our feature set 'X'
model.fit(X)

print("...Model training complete.")


# --- Step 4: Save the Trained Model ---
# We save the model to a file so our application can use it later
joblib.dump(model, 'fraud_model.joblib')
print("Model saved to 'fraud_model.joblib'")


# --- Step 5: Evaluate the Model's Performance ---
print("\n--- Model Evaluation ---")
# Let's see how well our model did. We'll make predictions on the whole dataset.
predictions = model.predict(X)

# The model outputs -1 for anomalies (fraud) and 1 for normal transactions.
# We need to convert this to our format (1 for fraud, 0 for normal).
pred_labels = np.where(predictions == -1, 1, 0)

# Now we compare the model's predictions (pred_labels) with the true labels (y)
report = classification_report(y, pred_labels, target_names=['Normal (0)', 'Fraud (1)'])

print("Classification Report:")
print(report)


Loading data and creating features...
...Data loaded successfully.

Training the Isolation Forest model...


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


...Model training complete.
Model saved to 'fraud_model.joblib'

--- Model Evaluation ---
Classification Report:
              precision    recall  f1-score   support

  Normal (0)       0.99      0.99      0.99      1000
   Fraud (1)       0.78      0.78      0.78        40

    accuracy                           0.98      1040
   macro avg       0.88      0.88      0.88      1040
weighted avg       0.98      0.98      0.98      1040

